# Proyecto: Visualización 3D de modelo de exposición

Github: [Github del proyecto](https://github.com/drodrguez/Visualizacion3D/)

## Descripción del proyecto

Este proyecto tiene como objetivo investigar e implementar metodologías novedosas de visualización de datos de exposición a distintas escalas espaciales, que permitan representar y comunicar de manera atractiva la información contenida en ellos, como ubicación, altura, densidad, materialidad, tipo de uso, edad, tipología estructural, etc.

## Datos disponibles

- Modelo de exposición: Los datos principales de este proyecto corresponde al modelo de exposición producto de la publicación "Development of national and local exposure models of residential structures in Chile". Este modelo se constituye de 1 archivo .txt para cada región, que incluye un catastro a nivel de bloque censal para 18 tipologías distintas de estructuras. 

- Datos de salud pública: Se obtienen directamente desde el [MINSAL](https://deis.minsal.cl/). Estos datos presentan el desafío de que son actualizados constantemente, por lo que su url es variable. Se debe pensar en como solucionar este problema.

- Datos de educación: Obtenidos desde la página de datos abiertos del [MINEDUC](http://datosabiertos.mineduc.cl/). Estos presentan el desafío de que su formato es en .rar, formato que es dificil de abordar desde la automatización, pues WinRar es un producto de licencia y no open-source.

- Datos de cartografía censal: Obtenidos desde la página del [censo](http://www.censo2017.cl/servicio-de-mapas/). Presenta el mismo problema: vienen en formato .rar.


## Carga de los datos.

### Modelo de exposición. 

Los modelos de exposición se encuentran en formato .txt, y deben ser transformados a un formato adecuado para el trabajo de datos geoespaciales. Esto se realiza a través de pandas y los métodos implementados en geopandas, que nos permiten obtener la geometría directamente desde un dataset.

In [39]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datashader as ds

In [2]:
# Obtenemos los path de los modelos de exposición por región almacenados en el github del proyecto.

exposure_paths = ["https://raw.githubusercontent.com/drodrguez/Visualizacion3D/main/Datos/Exposure_model/Exposure%20Model%20Block%20Reg%20"+str(num)+"_opt.txt" for num in range(1, 16) if num!= 13]
capital_exposure = ["https://raw.githubusercontent.com/drodrguez/Visualizacion3D/main/Datos/Exposure_model/ExposureModelBlockReg_13_"+str(num)+"_opt.txt" for num in range(1, 4)]
exposure_paths = exposure_paths + capital_exposure

In [3]:
exposure_geodata = dict()
capital = []
for path in exposure_paths:    
    data = pd.read_csv(path, encoding='latin-1')
    reg_id = data.region_id[0]
    if reg_id == 13:
        capital.append(data)
    else:
        geodata = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.latitude, data.longitude))          
        exposure_geodata[geodata.region_id[0]] = geodata
    
capital_data = pd.concat([data for data in capital])
geodata = gpd.GeoDataFrame(capital_data, geometry=gpd.points_from_xy(capital_data.latitude, capital_data.longitude)) 
exposure_geodata[13] = geodata

In [10]:
exposure_geodata[13].head()

,longitude,latitude,region_id,region_name,municipality_id,municipality_name,census_block,typology,number_dwellings,number_buildings,...,replace_cost_per_building_area(LocalCurrency/m2),replace_cost_per_building(USD),replace_cost_per_building(LocalCurrency),total_replace_cost(USD),total_replace_cost(LocalCurrency),occupants_per_dwelling,occupants_per_building,total_occupants,is_urban,geometry
0,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,1,0.31,0.31,...,NaN,41646.56,NaN,13006.92,NaN,2.93,2.93,0.91,True,POINT (-33.43581 -70.63756)
1,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,2,9.01,0.11,...,NaN,2366932.21,NaN,265499.02,NaN,1.95,156.26,17.53,True,POINT (-33.43581 -70.63756)
2,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,3,48.18,0.22,...,NaN,6327080.83,NaN,1386948.00,NaN,1.95,427.67,93.75,True,POINT (-33.43581 -70.63756)
3,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,4,28.17,0.08,...,NaN,9503050.08,NaN,787276.10,NaN,1.95,661.62,54.81,True,POINT (-33.43581 -70.63756)
4,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,5,0.13,0.13,...,NaN,48037.54,NaN,6374.48,NaN,3.20,3.20,0.42,True,POINT (-33.43581 -70.63756)


In [12]:
exposure_geodata[13].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 541989 entries, 0 to 127140
Data columns (total 26 columns):
 #   Column                                            Non-Null Count   Dtype   
---  ------                                            --------------   -----   
 0   longitude                                         541989 non-null  float64 
 1   latitude                                          541989 non-null  float64 
 2   region_id                                         541989 non-null  int64   
 3   region_name                                       541989 non-null  object  
 4   municipality_id                                   541989 non-null  int64   
 5   municipality_name                                 541989 non-null  object  
 6   census_block                                      541989 non-null  int64   
 7   typology                                          541989 non-null  int64   
 8   number_dwellings                                  541989 non-null 

In [14]:
exposure_geodata[13].describe()

,longitude,latitude,region_id,municipality_id,census_block,typology,number_dwellings,number_buildings,avg_dwellings_per_building,avg_number_storeys,...,total_area(m2),replace_cost_per_building_area(USD/m2),replace_cost_per_building_area(LocalCurrency/m2),replace_cost_per_building(USD),replace_cost_per_building(LocalCurrency),total_replace_cost(USD),total_replace_cost(LocalCurrency),occupants_per_dwelling,occupants_per_building,total_occupants
count,541989.000000,541989.000000,541989.0,541989.000000,5.419890e+05,541989.000000,541989.000000,541989.000000,541989.000000,467902.000000,...,467902.000000,467902.000000,0.0,5.419890e+05,0.0,5.419890e+05,0.0,541989.000000,541989.000000,541989.000000
mean,-70.680071,-33.499503,13.0,13172.426044,1.317250e+13,8.723915,3.857092,2.621032,29.247211,4.157491,...,372.845559,268.625945,NaN,6.798035e+05,NaN,1.009639e+05,NaN,3.317381,81.925327,12.660172
std,0.122039,0.103160,0.0,123.610325,1.236017e+11,5.330434,10.516034,6.505172,77.449957,5.658901,...,989.283857,102.689981,NaN,1.829612e+06,NaN,4.024971e+05,NaN,0.478383,210.114674,31.919286
min,-71.622740,-34.148550,13.0,13101.000000,1.310101e+13,1.000000,0.010000,0.000000,1.000000,1.000000,...,0.170000,93.200000,NaN,1.280000e+03,NaN,6.470000e+00,NaN,1.000000,1.000000,0.010000
25%,-70.734660,-33.560090,13.0,13111.000000,1.311102e+13,5.000000,0.130000,0.010000,1.000000,1.690000,...,13.980000,202.600000,NaN,1.169692e+04,NaN,9.870400e+02,NaN,2.990000,3.530000,0.410000
50%,-70.665650,-33.495180,13.0,13120.000000,1.312008e+13,7.000000,0.750000,0.230000,1.000000,1.980000,...,84.520000,214.480000,NaN,2.572208e+04,NaN,1.348897e+04,NaN,3.480000,3.740000,2.610000
75%,-70.600500,-33.435390,13.0,13131.000000,1.313105e+13,14.000000,3.790000,2.410000,27.560000,4.260000,...,370.150000,339.430000,NaN,4.984078e+05,NaN,7.585430e+04,NaN,3.690000,81.580000,12.760000
max,-69.995880,-32.932340,13.0,13605.000000,1.360599e+13,18.000000,820.140000,510.520000,660.000000,40.000000,...,79173.030000,544.600000,NaN,1.359193e+07,NaN,4.311764e+07,NaN,5.250000,1846.000000,2885.290000


Surgen algunas dudas:

- ¿Por qué los valores de ```number_dwellings```, ```number_buildings```, y ```occupants_per_dwelling``` adquieren esos valores tan particulares? Son decimales, ¿deberían ser aproximados? ¿Cómo corregimos esta anomalía? ¿Por qué objetos discretos están representados de forma contínua? 

### Establecimientos de salud

In [15]:
link_salud = 'https://repositoriodeis.minsal.cl/DatosAbiertos/Establecimientos_ChileDEIS_MINSAL%2005-08-2022.xlsx'

In [18]:
salud_data = pd.read_excel(link_salud, sheet_name='Establecimientos Vig')

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell U4348 is marked as a date but the serial value 973784866 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [41]:
salud_data.head()

SyntaxError: unmatched ')' (292141053.py, line 1)

In [24]:
salud_data.columns = salud_data.iloc[0,:]

In [26]:
salud_data.columns

Index([                                           'Código Antiguo ',
                                                   'Código Vigente',
                                                    'Código  Madre',
                                               'Código Nuevo Madre',
                                                    'Código Región',
                                                    'Nombre Región',
       'Código Dependencia Jerárquica (SEREMI / Servicio de Salud)',
       'Nombre Dependencia Jerárquica (SEREMI / Servicio de Salud)',
                                              'Pertenencia al SNSS',
                                             'Tipo Establecimiento',
                                         'Ámbito de Funcionamiento',
                                                   'Nombre Oficial',
                                                   'Certificación ',
                                       'Dependencia Administrativa',
                                  

In [40]:
salud_data.iloc[1:, :-3].head()

,Código Antiguo,Código Vigente,Código Madre,Código Nuevo Madre,Código Región,Nombre Región,Código Dependencia Jerárquica (SEREMI / Servicio de Salud),Nombre Dependencia Jerárquica (SEREMI / Servicio de Salud),Pertenencia al SNSS,Tipo Establecimiento,...,Tiene Servicio de Urgencia,Tipo de Urgencia,Tipo de SAPU,LATITUD [Grados decimales],LONGITUD [Grados decimales],Tipo de Prestador Sistema de Salud,Estado de Funcionamiento,Nivel de Complejidad,Modalidad de Atención,Fecha de Incorporación a la base o cambios
1,26-704,126704,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,SI,Urgencia Hospitalaria (UEH),No Aplica,-54.935209,-67.600393,Público,Vigente en operación,Baja Complejidad,Atención Cerrada-Hospitalaria,NaN
2,26-204,126204,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,SI,Urgencia Hospitalaria (UEH),No Aplica,-54.934374,-67.608895,Pendiente,Vigente en operación,Mediana Complejidad,Atención Cerrada-Hospitalaria,NaN
3,26-412,126412,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,NO,No Aplica,No Aplica,-53.640546,-69.645848,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN
4,26-414,126414,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,NO,No Aplica,No Aplica,-53.404244,-70.990158,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN
5,26-102,126102,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,SI,Urgencia Hospitalaria (UEH),No Aplica,-53.298162,-70.358384,Público,Vigente en operación,Baja Complejidad,Atención Cerrada-Hospitalaria,NaN


Recordemos que esta base de datos es variable, se actualiza constantemente y no tenemos la seguridad de que las columnas, tanto en su distribución como en nombre, vayan a conservar su formato para actualizaciones futuras. Luego, se hace necesario:

- 1. Reducir las columnas para conservar sólo la información necesaria para el proyecto.
- 2. Automatizar la búsqueda de estas columnas de utilidad, mediante palabras clave.

¿Qué información nos es de utilidad, entonces?

- Datos de geolocalización: latitud y longitud.

- Tipo de establecimiento: esta base de datos cuenta con diversos tipos de establecimiento: hospitales, sapu, hasta incluso centros de cirugía estética. 

- Nivel de complejidad: En el contexto del proyecto, tiene lógica asumir que, si existe compromiso de los establecimientos de mayor complejidad, el acceso a la salud, sobretodo en caso de desastres, se ve más comprometida v/s un establecimiento de baja complejidad. 

Para esto haremos un sondeo de los valores que pueden tomar estas columnas, para filtrar según lo requerido.